In [ ]:
!pip install pandas
!pip install mysql-connector-python
!pip install openpyxl

#Inicializar docker desde la consola: sudo dockerd

In [ ]:
import pandas as pd
import mysql.connector

## CARGAR EL PRIMER DATASET EN STAGING

In [ ]:
df = pd.read_csv('../data/raw/ISCED_2013.csv', encoding ='latin-1', delimiter=';')

In [ ]:
df.head()

In [ ]:
columns = df.columns
new_columns_dict = {}
cod_count = 1
nom_count = 1

for col in columns:
    if 'COD_' in col:
        new_columns_dict[col] = f'codigo_rama_{cod_count}'
        cod_count += 1
    elif 'NOM_' in col:
        new_columns_dict[col] = f'nombre_rama_{nom_count}'
        nom_count += 1
    else:
        new_columns_dict[col] = col

# Use the rename() method to rename the columns
df.rename(columns=new_columns_dict, inplace=True)

# Print the updated DataFrame
df.head(2)

In [ ]:
cnx =mysql.connector.connect(
    host='localhost',
    user='admin',
    password='admin',
    database='dw',
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()

In [ ]:
data = [tuple(row) for _, row in df.iterrows()]

query=f"INSERT INTO stage_ramas_conocimiento VALUES ({','.join(['%s'] * len(df.columns))})"

cursor.executemany(query, data)

try:
    cnx.commit()
    print("Datos insertados de manera exitosa!")
except Exception as e:
    print("Error al insertar los datos")
    print("Error code:", e.errno)
    print("Error description:", e.strerror)

In [ ]:
query=f"SELECT * FROM stage_ramas_conocimiento"

df_2 = pd.read_sql(query, cnx)

df_2.head(2)

cursor.close()

In [ ]:
#mysql -u admin -p

## CARGAR EL SEGUNDO DATASET EN STAGING

In [ ]:
#Read data from SEGR1.csv SEGR2.csv files
df_1 = pd.read_csv('../data/raw/SEGR1.csv', encoding ='latin-1', delimiter=';')
df_2 = pd.read_csv('../data/raw/SEGR2.csv', encoding ='latin-1', delimiter=';')

In [ ]:
df_union = pd.concat([df_1, df_2])
df_union.head(2)

In [ ]:
data_cols = df_union.columns

id_columns = [col for col in data_cols if not 'EGR_' in col]
data_columns = [col for col in data_cols if 'EGR_' in col]

In [ ]:
#Consolidar información de todos los años en una sola columna
df_unpivot = pd.melt(df_union, id_vars=id_columns, value_vars=data_columns, var_name='año', value_name='num_egresados')

#Actualizar columna de año usando los últimos dos digitos del encabezado
df_unpivot['año'] = '20' + df_unpivot['año'].str[-2:]

df_unpivot['pais'] = 'España'

In [ ]:
#Validar resultados de la columna año
df_unpivot['año'].value_counts()

In [ ]:
df_unpivot.rename(columns={'UNIVERSIDAD':'nombre_universidad', 'TIPO_UNIVERSIDAD': 'tipo_universidad', 'MODALIDAD': 'modalidad', 'RAMA_ENSEÑANZA':'rama_enseñanza'}, inplace=True)

In [ ]:
df_unpivot = df_unpivot[['año','pais','nombre_universidad','tipo_universidad','modalidad','rama_enseñanza','num_egresados']]
df_unpivot.head(2)

In [ ]:
df_unpivot.tipo_universidad.value_counts()

In [ ]:
len(df_unpivot['nombre_universidad'].drop_duplicates())

In [ ]:
len(df_unpivot['rama_enseñanza'].drop_duplicates())

In [ ]:
df_unpivot.info()

In [ ]:
cursor = cnx.cursor()

data = [tuple(row) for _, row in df_unpivot.iterrows()]

query=f"INSERT INTO stage_egresados_universidad VALUES ({','.join(['%s'] * len(df_unpivot.columns))})"

cursor.executemany(query, data)

try:
    cnx.commit()
    print("Datos insertados de manera exitosa!")
except Exception as e:
    print("Error al insertar los datos")
    print("Error code:", e.errno)
    print("Error description:", e.strerror)

In [ ]:
query=f"SELECT * FROM stage_egresados_universidad"

df_2 = pd.read_sql(query, cnx)

df_2.head(2)

In [ ]:
cursor.close()

## CARGAR EL TERCER DATASET EN STAGING

In [ ]:
df_1 = pd.read_excel('../data/raw/educ_uoe_grad01.xlsx', sheet_name='Data', skiprows=11, nrows=13)
df_2 = pd.read_excel('../data/raw/educ_uoe_grad01.xlsx', sheet_name='Data2', skiprows=11, nrows=13)
df_3 = pd.read_excel('../data/raw/educ_uoe_grad01.xlsx', sheet_name='Data3', skiprows=11, nrows=13)

In [ ]:
df_1.drop('Unnamed: 6', inplace=True, axis=1)

In [ ]:
#Consolidar información de todos los años en una sola columna
df1_unpivot = pd.melt(df_1, id_vars=['GEO/TIME'], value_vars=['2013','2014','2015','2016','2017'], var_name='year', value_name='num_graduated')

#Consolidar información de todos los años en una sola columna
df2_unpivot = pd.melt(df_2, id_vars=['GEO/TIME'], value_vars=['2013','2014','2015','2016','2017'], var_name='year', value_name='num_graduated_M')

#Consolidar información de todos los años en una sola columna
df3_unpivot = pd.melt(df_3, id_vars=['GEO/TIME'], value_vars=['2013','2014','2015','2016','2017'], var_name='year', value_name='num_graduated_F')

In [ ]:
merge_df = df1_unpivot.merge(df2_unpivot, on=['GEO/TIME','year'] , how='left')
merge_df = merge_df.merge(df3_unpivot, on=['GEO/TIME','year'], how='left')
merge_df.head(2)

In [ ]:
merge_df.rename(columns={'GEO/TIME': 'country'}, inplace=True)

#Cambiar el orden de las columnas
merge_df = merge_df[['year','country','num_graduated_M','num_graduated_F','num_graduated']]

merge_df.replace(':', 0, inplace=True)

In [ ]:
merge_df.country.value_counts()

In [ ]:
cursor = cnx.cursor()

data = [tuple(row) for _, row in merge_df.iterrows()]

query=f"INSERT INTO stage_numero_egresados_internacional VALUES ({','.join(['%s'] * len(merge_df.columns))})"

cursor.executemany(query, data)

try:
    cnx.commit()
    print("Datos insertados de manera exitosa!")
except Exception as e:
    print("Error al insertar los datos")
    print("Error code:", e.errno)
    print("Error description:", e.strerror)

In [ ]:
query=f"SELECT * FROM stage_numero_egresados_internacional"

df_2 = pd.read_sql(query, cnx)

df_2.head(2)

In [ ]:
cursor.close()

In [ ]:
cnx.close()